<a href="https://colab.research.google.com/github/interstellxr/deshima-atmfit/blob/main/atmfit-v1.0.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Import libraries & dependencies

# standard library
from typing import Any, Tuple

# dependencies
import casatools
import numpy as np
import matplotlib.pyplot as plt

# for xarray
import pandas as pd
import decode as dc
import xarray as xr

# for the fitting function
from scipy.optimize import least_squares
from scipy.interpolate import interp1d
from astropy.io import fits

In [ ]:
# @title ATM model (Pardo et al. 2001)

def get_tau(
    f_min: float,
    f_max: float,
    f_step: float,
    pwv: float,
    **atm_params: Any,
) -> Tuple[np.ndarray, np.ndarray]:
    """Compute of zenith opacities at given frequencies.

    Args:
        f_min: Minimum frequency (in units of Hz).
        f_max: Maximum frequency (in units of Hz).
        f_step: Frequency step (in units of Hz).
        pwv: Precipitable water vapor (in units of mm).
        atm_params: Parameters fo the ATM model.

    Returns:
        freq: Array of frequencies (in units of Hz).
        tau: Array of zenith opacities at the frequencies.

    """
    at = casatools.atmosphere()
    qa = casatools.quanta()

    f_cent = qa.quantity((f_min + f_max) / 2, "Hz")
    f_width = qa.quantity(f_max - f_min + f_step, "Hz")
    f_step = qa.quantity(f_step, "Hz")
    pwv = qa.quantity(pwv, "mm")

    at.initAtmProfile(**atm_params)
    at.initSpectralWindow(1, f_cent, f_width, f_step)
    at.setUserWH2O(pwv)

    freq = qa.convert(at.getSpectralWindow(), "Hz")["value"]
    tau = at.getDryOpacitySpec()[1] + at.getWetOpacitySpec()[1]['value']

    return freq, tau

In [ ]:
# @title Download filter responses (DDB)

! gdown "1gE0IJzlJpN9xrCqXSOvg7AJw1GPAxYhK"
ddb = fits.open("ddb_20231123.fits.gz")

In [ ]:
# @title Main function

def ATM_fit(da, pwv0, dt, *ranges_weights): # Obligatory input : DEMS (xarray.DataArray)
# Optional inputs : initial PWV value (float), time step (int), weighted frequency ranges (tuples of floats : (fmin, fmax, weight))


    ## ERROR HANDLING ##

    ranges = []
    weights = []

    for r_w in ranges_weights:
        if len(r_w) != 3:
            raise ValueError("Each range must contain minimum and maximum frequencies and have a corresponding weight.")
        ranges.append(r_w[0:2])
        weights.append(r_w[-1])

    if not isinstance(da, xr.DataArray):
        raise TypeError(f"Argument '{da}' must be an xarray DataArray, got {type(da).__name__} instead.")

    if not isinstance(ddb, fits.HDUList):
        raise TypeError(f"Argument '{ddb}' must be a .fits, got {type(ddb).__name__} instead.")

    for k,frange in enumerate(ranges):
        w = weights[k]
        for arg in [frange[0], frange[1], pwv0, w]:
            if arg is not None and not isinstance(arg, (int, float)):
                raise TypeError(f"Argument '{arg}' must be a number, got {type(arg).__name__} instead.")
            if arg is not None and arg is not pwv0 and arg is not w and arg <= 0:
                raise ValueError(f"Argument '{arg}' must be a strictly positive number.")
            if (arg is pwv0 or arg is w) and arg is not None and arg < 0:
                raise ValueError(f"Argument '{arg}' must be a positive number or null.")

        if k != 0 and k!= len(ranges)-1:
            if frange[0] is None or frange[1] is None:
                raise ValueError(f"Both fmin and fmax must be specified.")

        if frange[0] is not None and frange[1] is not None:
            if frange[0] > frange[1]:
                raise ValueError(f"fmin must be less than or equal to fmax.")

    if dt is not None and dt >= len(da.time.values):
        print("Warning: dt is greater than time series length, which might not be intended.")

    if dt is not None and ( not isinstance(dt, int) or dt<=0 ) :
        raise TypeError(f"dt must be a positive integer, got {type(dt).__name__} instead.")


    ## CREATE ATM PARAMETERS ##

    def get_first_non_nan(arr, default):
        for value in arr:
            if not np.isnan(value):
                return value
        return default

    default_pressure = 570  # mbar
    default_humidity = 20  # percent
    default_temperature = 0  # C

    pressure_value = get_first_non_nan(da.pressure.values, default_pressure)
    humidity_value = get_first_non_nan(da.humidity.values, default_humidity)
    temperature_value = get_first_non_nan(da.temperature.values, default_temperature) + 273.15 # convert to Kelvin (DEMS temperatures are in C)

    atm_params = {
        'atmType': 1,
        'humidity': humidity_value,
        'temperature': f'{temperature_value} K',
        'altitude': '4860 m',
        'pressure': f'{pressure_value} mbar',
        'h0': '2.0 km',
    }


    ## LOAD DATA ##

    T_atmos = temperature_value

    freq = da.d2_mkid_frequency.values
    freq_unsorted = freq

    kidfilt = ddb["KIDFILT"] # fits file
    masterid = kidfilt.data["masterid"] # channel IDs of filter response functions (more than in DEMS)
    nu = kidfilt.data["Raw Toptica F"] # corresponding frequencies (same for all IDs normally)
    R = kidfilt.data["Raw df resp."] # corresponding responses (y data)

    common_indices = np.where(np.isin(masterid, da.chan.values))[0] # find common channels
    nu = nu[common_indices]
    R = R[common_indices]

    sorted_indices = np.argsort(freq) # sort frequencies (just for simplicity in the following lines)
    freq = freq[sorted_indices]
    nu = nu[sorted_indices]
    R = R[sorted_indices]

    for range in ranges: # check given ranges are subsets of global range
        fmin = range[0]
        fmax = range[1]
        if (fmin is not None and fmin < freq[0]) :
            raise ValueError(f"fmin must be within the frequency range.")

        if (fmax is not None and fmax > freq[-1]) :
            raise ValueError(f"fmax must be within the frequency range.")

    N = len(freq)-1
    m = freq[0]
    M = freq[-1]
    step = (freq[-1] - freq[0])/N # we want a step as close to the data as possible

    # set default values to arguments
    if pwv0 is None:
        init_PWV = 1.0
    else :
        init_PWV = pwv0
    if dt is None:
        time_step = 1
    else :
        time_step = dt
    if w is None:
        W = 1
    else :
        W = w


    ## FITTING FUNCTION ##

    def Tb_fit(t,freq,fit_T_atm=True, T_atm=None): # take in boolean to see if we should fit for T_atm (only for t==0)


        ## LOAD DATA AND HANDLE NANS ##

        Tb = da[t,:].values
        Tb= Tb[sorted_indices]
        valid_indices = np.where(~np.isnan(Tb))
        Tb_valid = Tb[valid_indices]
        freq_valid = freq[valid_indices]

        if not np.isnan(da.secz.values[t]):
            airmass = da.secz.values[t]

        else : # airmass is the last non nan value of secz before t or first non nan value after t (whichever is closest to t)
            before_t_idx = np.where(~np.isnan(da.secz_values[:t]))[0] # last non nan before t
            if before_t_idx.size > 0:
                last_idx = before_t_idx[-1]
                last = da.secz_values[last_idx]
            else:
                last_idx = None
                last = np.nan

            after_t_idx = np.where(~np.isnan(da.secz_values[t+1:]))[0] + t + 1 # first non nan after t
            if after_t_idx.size > 0:
                first_idx = after_t_idx[0]
                first = da.secz_values[first_idx]
            else:
                first_idx = None
                first = np.nan

            if last_idx is not None and first_idx is not None:
                if (t - last_idx) <= (first_idx - t): # determine which is closest
                    airmass = last
                else:
                    airmass = first
            elif last_idx is not None:
                airmass = last
            elif first_idx is not None:
                airmass = first_idx
            else:
                airmass = 1 # default if all are NaNs


        # create an empty array with the same shape as Tb : useful for ouput (we want to keep NaNs to match input)
        empty_Tb = np.empty_like(Tb, dtype = 'float')
        empty_Tb[np.isnan(Tb)] = np.nan

        # R is an array of response functions (themselves arrays with x values being frequencies)
        # so we iterate over the responses and interpolate to match frequencies
        def interpolate_responses(frequencies, responses, freq_valid):
            return np.array([interp1d(freq, resp, kind='linear', fill_value="extrapolate")(freq_valid) for freq, resp in zip(frequencies, responses)])

        filter_responses = interpolate_responses(nu[valid_indices], R[valid_indices], freq_valid)


        ## CONVERT OPACITY GIVEN BY MODEL TO SKY BRIGHTNESS TEMPERATURE ##

        def tau_to_T(params):
            pwv, T_atm_ = params if fit_T_atm else (params[0], T_atm)
            temp, tau = get_tau(m*1e9, M*1e9, step*1e9, pwv, **atm_params)
            tau = interp1d(temp[valid_indices], tau[valid_indices], kind='linear', bounds_error=False, fill_value="extrapolate")(freq_valid*1e9)
            T_model = T_atm_ * (1 - np.exp(-tau*airmass))

            norm = np.sum(filter_responses, axis=0)
            filtered_T_model = np.sum([T_model * resp for resp in filter_responses], axis=0) / norm

            return filtered_T_model


        ## RESIDUAL FUNCTION ##

        def diff(params, w):
            return abs(Tb_valid - tau_to_T(params)) * w

        weights_array = np.ones_like(Tb_valid)

        for r, W in zip(ranges,weights): # assign weights
            if r[0] is None and r[1] is not None:
                mask = (freq_valid >= freq_valid[0]) & (freq_valid <= r[1])
            if r[1] is None and r[0] is not None:
                mask = (freq_valid >= r[0]) & (freq_valid <= freq_valid[-1])
            if r[0] is None and r[1] is None:
                mask = (freq_valid >= freq_valid[0]) & (freq_valid <= freq_valid[-1])
            if r[0] is not None and r[1] is not None:
                mask = (freq_valid >= r[0]) & (freq_valid <= r[1])
            weights_array[np.where(mask)] = W


        ## LEAST SQUARES FIT ##

        init_params = [init_PWV, T_atm] if fit_T_atm else [init_PWV]
        bounds = ([0, 200], [np.inf, 300]) if fit_T_atm else ([0], [np.inf]) # bounds for PWV and T_atm
        result = least_squares(diff,init_params,loss='huber', kwargs={'w': weights_array}, bounds = bounds)

        optim_PWV, optim_T_atm = result.x if fit_T_atm else (result.x[0], T_atm)

        J = result.jac
        cov = np.linalg.inv(J.T.dot(J))
        std = np.sqrt(np.diagonal(cov))
        PWV_std, T_atm_std = std if fit_T_atm else (std[0], 0)

        Tmodel = tau_to_T(result.x)

        # Re-insert NaN positions
        k=0
        for i,el in enumerate(empty_Tb):
              if not np.isnan(el):
                  empty_Tb[i] = Tmodel[k]
                  k+=1

        unsorted_indices = np.argsort(sorted_indices) # unsort indices to match input data array
        empty_Tb = empty_Tb[unsorted_indices]

        return empty_Tb, optim_PWV, optim_T_atm, PWV_std, T_atm_std # return unsorted and invalid (with NaNs) Tb_model and optimized PWV and optimized T_atm with errors


    ## LOOP OVER TIME ##

    init_time = 0
    _, _, initial_T_atm, _, initial_T_atm_std = Tb_fit(init_time, freq, fit_T_atm=True, T_atm=T_atmos)

    time_values=pd.Series(da.time.values[0:len(da.time):time_step])

    Tb_model_array = []
    optim_PWV_array = np.array([])
    optim_T_atm_array = np.array([])
    PWV_std_array = np.array([])
    T_atm_std_array = np.array([])

    for time_str in time_values:
        t = np.where(da.time.values == time_str)[0][0]
        T, PWV, T_ATM, PWV_std, _ = Tb_fit(t,freq, fit_T_atm=False,T_atm=initial_T_atm)
        Tb_model_array.append(T)
        optim_PWV_array = np.append(optim_PWV_array,PWV)
        optim_T_atm_array = np.append(optim_T_atm_array, T_ATM)
        PWV_std_array = np.append(PWV_std_array, PWV_std)
        T_atm_std_array = np.append(T_atm_std_array, initial_T_atm_std)


    ## CREATE THE OUTPUT DATA ARRAY ##

    new_da = da
    new_da.values = np.array(Tb_model_array)
    new_da.assign_coords(PWV=("time",optim_PWV_array))
    new_da.assign_coords(T_atm=("time",optim_T_atm_array))
    new_da.assign_coords(T_atm_std=("time",T_atm_std_array))
    new_da.assign_coords(PWV_std=("time",PWV_std_array))

    new_da['PWV'].attrs['long name'] = 'Precipitable Water Vapor derived from ATM model'
    new_da['PWV'].attrs['units'] = 'mm'

    new_da['T_atm'].attrs['long_name'] = 'Atmospheric Temperature derived from ATM model'
    new_da['T_atm'].attrs['units'] = 'K'

    new_da['PWV_std'].attrs['long_name'] = 'Standard deviation of PWV derived from ATM model'
    new_da['PWV_std'].attrs['units'] = 'mm'

    new_da['T_atm_std'].attrs['long_name'] = 'Standard deviation of T_atm derived from ATM model'
    new_da['T_atm_std'].attrs['units'] = 'K'

    return new_da